In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt

In [2]:
stations = pd.read_csv('../src/Station Master List.csv')

Use glob to read in all 2018 and 2019 data. I tried loading in more, and got an encoding error. I'll ignore it for now and keep going

In [3]:
trips = pd.concat([pd.read_csv(f) for f in glob.glob('../src/Trip Data/*201[8-9].csv')], ignore_index = True)

Join the station spatial data for the locations with the trip data. This will give a start and stop location for each trip

In [4]:
checkout_spatial = trips.set_index('CheckoutKioskName')\
    .join(stations.set_index('Station Name')[['Latitude', 'Longitude', 'Dock']])\
    .rename(columns={'Latitude': 'checkout_latitude', 'Longitude':'checkout_longitude', 'Dock': 'checkout_dock'})\
    .reset_index()

return_spatial = trips[['ReturnKioskName']].set_index('ReturnKioskName')\
    .join(stations.set_index('Station Name')[['Latitude', 'Longitude', 'Dock']])\
    .rename(columns={'Latitude': 'return_latitude', 'Longitude':'return_longitude', 'Dock': 'return_dock'})\
    .reset_index()

joined_trips = pd.concat([checkout_spatial, return_spatial], axis=1)

Get the checkout date and time into one column that is a pandas datetime datatype.

In [5]:
joined_trips['CheckoutDateTime'] = pd.to_datetime(joined_trips['CheckoutDateLocal']\
                                                  + ' ' + \
                                                  joined_trips['CheckoutTimeLocal'])

The names of member types have changed over time. It will be useful to combine all of the annual membership holders into one key and all of the single or day use pass holders into another key

In [6]:
annual_member_types = ['Annual Member', 'Annual Membership', 'Annual', 'Annual Pass', 'Annual Plus']
single_use_or_day_pass = ['Single Use Pass', '24 Hour Pass', 'Single Ride 24']

membership_type_map = dict()
for i in annual_member_types:
    membership_type_map[i] = 'Annual'

for i in single_use_or_day_pass:    
    membership_type_map[i] = 'Single/Day'
    
joined_trips['agg_member_type'] = joined_trips['MembershipType'].map(membership_type_map) 

In [7]:
joined_trips.to_csv('../cleaned_data/2018-2019_trips.csv', index=False)
joined_trips.to_pickle('../cleaned_data/2018-2019_trips')

In [8]:
joined_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410000 entries, 0 to 409999
Data columns (total 37 columns):
index                      410000 non-null object
TripId                     410000 non-null int64
UserProgramName            410000 non-null object
UserId                     410000 non-null int64
UserRole                   410000 non-null object
UserCity                   113843 non-null object
UserState                  113749 non-null object
UserZip                    354847 non-null object
UserCountry                410000 non-null object
MembershipType             378884 non-null object
Bike                       410000 non-null object
BikeType                   410000 non-null object
ReturnKioskName            410000 non-null object
DurationMins               410000 non-null int64
AdjustedDurationMins       410000 non-null int64
UsageFee                   410000 non-null float64
AdjustmentFlag             410000 non-null object
Distance                   410000 non-null